# Neural Network Predictions

### Imports and Setup

In [1]:
import numpy as np
import csv
from skopt import gp_minimize
from skopt.space import Real, Integer
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, Input, SimpleRNN # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore

from NeuralNetworks.PreProcessing import preprocess_data, learning_set
from NeuralNetworks.PostProcessing import plot_test_and_prediction
from NeuralNetworks.LearningInstance import LearningInstance
from NeuralNetworks.Predictions import predict

In [2]:
n = 113
T = 118
q = 31
q0 = 16

names_path = "Data/names.txt"
gdp_path = "Data/yp_raw.csv"

### Loading Data

In [3]:
names = []
with open(names_path, 'r') as file:
    rows = file.readlines()
    for row in rows:
        names.append(row[:3])

In [4]:
gdp = np.zeros((n,T))
with open(gdp_path, 'r') as file:
    rows = csv.reader(file)
    for i, row in enumerate(rows):
        for j, val in enumerate(row):
            gdp[j][i] = float(val)

### Data Pre-Processing

In [5]:
log_gdp, low_gdp = preprocess_data(gdp, T, q, q0)


### Neural Network Predictions

In [6]:
lags = 10
split = 0.7
horizon = 50
learning_sets: list[LearningInstance] = learning_set(
    lags, split, gdp, log_gdp, low_gdp, names
)

In [7]:
space = [Real(1e-6, 1e-2, "log-uniform", name='learning_rate'),
         Integer(1, 5, name='num_layers'),
         Integer(10, 500, name='num_units')]

In [ ]:
country = learning_sets[0]
usa = learning_sets[names.index("USA")]
print(country.country)

In [9]:
def build_MLP(learning_rate, num_layers, num_units, lags):
    model = Sequential()
    model.add(Input(shape=(lags,)))

    for i in range(num_layers):
        model.add(Dense(int(num_units), activation='relu'))
    
    model.add(Dense(1))

    model.compile(
        optimizer=Adam(learning_rate=learning_rate), 
        loss = 'mean_squared_error'
    )

    return model

In [10]:
def build_RNN(learning_rate, num_layers, num_units, lags):
    model = Sequential()
    
    model.add(Input(shape=(lags,1)))

    # Add LSTM layers
    for i in range(num_layers):
            model.add(SimpleRNN(int(num_units), return_sequences=i<num_layers-1))
    
    # Output layer for regression
    model.add(Dense(1, activation='linear'))

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='mean_squared_error'
    )

    return model

In [11]:
def objective_MLP(params):
    learning_rate, num_layers, num_units = params
    num_units = int(num_units)
    model = build_MLP(learning_rate, num_layers, num_units, lags)
    history = model.fit(
        country.x_train, 
        country.y_train, 
        validation_data = (country.x_test, country.y_test), 
        epochs = 50, 
        batch_size = 32,
        verbose = 0
    )
    val_loss = history.history['val_loss'][-1]
    # Return the validation loss (or any metric to minimize)
    return val_loss

In [12]:
def objective_RNN(params):
    learning_rate, num_layers, num_units = params
    num_units = int(num_units)
    model = build_RNN(learning_rate, num_layers, num_units, lags)
    history = model.fit(
        country.x_train.reshape(country.x_train.shape[0], country.x_train.shape[1], 1), 
        country.y_train, 
        validation_data = (country.x_test.reshape(country.x_test.shape[0], country.x_test.shape[1],1), country.y_test), 
        epochs = 50, 
        batch_size = 32,
        verbose = 0
    )
    val_loss = history.history['val_loss'][-1]
    # Return the validation loss (or any metric to minimize)
    return val_loss

In [13]:
result_mlp = gp_minimize(objective_MLP, space, n_calls=50, random_state=0)

In [ ]:
print(result_mlp.x)

In [15]:
optimized = build_MLP(result_mlp.x[0], result_mlp.x[1], result_mlp.x[2], lags)

In [16]:
history = optimized.fit(
    country.x_train, 
    country.y_train, 
    validation_data = (country.x_test, country.y_test), 
    epochs = 50, 
    batch_size = 32,
    verbose = 0
)

In [ ]:
loss = optimized.evaluate(country.x_test, country.y_test, verbose=0)
test_predictions = optimized.predict(country.x_test, verbose=0).T[0]
print(loss)

In [18]:
def predict_RNN(model: Sequential, 
            inst: LearningInstance, 
            lags: int, 
            horizon: int
            ) -> np.ndarray:
    """
    Performs iterative predictions for a given neural network

    Parameters:
    model (Sequential): The neural network with which predictions are performed
    inst (LearningInstance): The learning instance (country) to predict for
    lags (int): the amount of previous values to be considered
    horizon (int): the amount of future values to be predicted

    Returns:
    np.ndarray: The forecasted values
    """
    last = inst.low_freq[-lags:]
    future = np.zeros(horizon)
    for i in range(horizon):
        p = model.predict(last.reshape(1,lags,1), verbose=0)[0][0]
        future[i] = p
        last = np.roll(last,-1)
        last[-1] = p
    return future

In [19]:
future = predict(optimized, country, lags, horizon)

In [ ]:
plot_test_and_prediction(country, test_predictions, future, "MLP Predictions for ARG")

In [ ]:
history_usa = optimized.fit(
    usa.x_train, 
    usa.y_train, 
    validation_data = (usa.x_test, usa.y_test), 
    epochs = 50, 
    batch_size = 32,
    verbose = 0
)
loss_usa = optimized.evaluate(usa.x_test, usa.y_test, verbose=0)
test_predictions_usa = optimized.predict(usa.x_test, verbose=0).T[0]
print(loss_usa)
future_usa = predict(optimized, usa, lags, horizon)
plot_test_and_prediction(usa, test_predictions_usa, future_usa, "MLP predictions for USA (from ARG-tuned model)")

In [23]:
def objective_usa(params):
    learning_rate, num_layers, num_units = params
    num_units = int(num_units)
    model = build_MLP(learning_rate, num_layers, num_units, lags)
    history = model.fit(
        usa.x_train, 
        usa.y_train, 
        validation_data = (usa.x_test, usa.y_test), 
        epochs = 50, 
        batch_size = 32,
        verbose = 0
    )
    val_loss = history.history['val_loss'][-1]
    # Return the validation loss (or any metric to minimize)
    return val_loss

In [24]:
result_usa = gp_minimize(objective_usa, space, n_calls=50, random_state=0)

In [25]:
optimized_usa = build_MLP(result_usa.x[0], result_usa.x[1], result_usa.x[2], lags)

In [ ]:
history_usa1 = optimized_usa.fit(
    usa.x_train, 
    usa.y_train, 
    validation_data = (usa.x_test, usa.y_test), 
    epochs = 50, 
    batch_size = 32,
    verbose = 0
)
loss_usa1 = optimized_usa.evaluate(usa.x_test, usa.y_test, verbose=0)
test_predictions_usa1 = optimized_usa.predict(usa.x_test, verbose=0).T[0]
print(loss_usa1)
future_usa1 = predict(optimized_usa, usa, lags, horizon)
plot_test_and_prediction(usa, test_predictions_usa1, future_usa1, "MLP predictions for USA (from USA-tuned model)")

In [32]:
result_rnn = gp_minimize(objective_RNN, space, n_calls=50, random_state=0)

In [ ]:
print(result_rnn.x)

In [34]:
optimized_rnn = build_RNN(result_mlp.x[0], result_mlp.x[1], result_mlp.x[2], lags)

In [36]:
history_rnn = optimized_rnn.fit(
    country.x_train.reshape(country.x_train.shape[0], country.x_train.shape[1], 1), 
    country.y_train, 
    validation_data = (country.x_test.reshape(country.x_test.shape[0], country.x_test.shape[1],1), country.y_test), 
    epochs = 50, 
    batch_size = 32,
    verbose = 0
)

In [ ]:
loss_rnn = optimized_rnn.evaluate(country.x_test.reshape(country.x_test.shape[0], country.x_test.shape[1], 1), country.y_test, verbose=0)
test_predictions_rnn = optimized_rnn.predict(country.x_test.reshape(country.x_test.shape[0], country.x_test.shape[1], 1), verbose=0).T[0]
print(loss_rnn)

In [ ]:
future_rnn = predict_RNN(optimized_rnn, country, lags, horizon)
plot_test_and_prediction(country, test_predictions_rnn, future_rnn, "RNN prediction for ARG")